##### 데싸노트의 실전에서 통하는 머신러닝 (골든레빗 MUST HAVE 시리즈)
- 랜덤 포레스트 (Random Forest)
- 결정트리의 단점인 오버피팅을 개선한 트리 모델
- 랜덤으로 생성된 무수히 많은 트리를 이용해 예측을 수행함
- 이런 식으로 여러 모델을 활용해서 하나의 모델을 만드는 기법을 앙상블이라고 함

- 결정 트리와 마찬가지로 아웃라이어 영향을 거의 받지 않음
- 선형/비선형 데이터 관계없이 잘 작동함
- 학습 속도가 느리고, 수많은 트리를 동원해서 해석이 어려울 수 있음

##### 오늘은 중고차 가격 예측합니다

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
data = pd.read_csv(file_url)

In [2]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


- seller_type : 판매자 유형, owner : 차주 변경 내역, engine : 배기량

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


- 이슈1 : 결측치 두둥등장
- 이슈2 : engine은 숫자로 표현해도 될 거 같은데 굳이 object로?

In [4]:
round(data.describe(), 2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


- 이슈3 : 저 미친 아웃라이어 어떻게합니까 (selling_price, km_driven)
- 하지만 일단 아웃라이어는 무시해도 괜찮음. 오늘 쓸 건 랜덤포레스트니까

In [5]:
# 일단 엔진에 있는 CC 쳐냅시다

data['engine'].str.split(expand=True)

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [6]:
# 분할된 걸 컬럼에 다시 넣을겁니다

data[['engine', 'engine_unit']] = data['engine'].str.split(expand=True)

In [7]:
data['engine'].head()

0    1248
1    1498
2    1497
3    1396
4    1298
Name: engine, dtype: object

In [8]:
# 쓰읍.. 왜 object 타입이야 이건 또
# 바꿔!

data['engine'] = data['engine'].astype('float32')

In [9]:
data['engine'].head()

0    1248.0
1    1498.0
2    1497.0
3    1396.0
4    1298.0
Name: engine, dtype: float32

In [10]:
# 아까 쪼갰던 다른 파트 확인해볼까요

data['engine_unit'].unique()

array(['CC', nan], dtype=object)

In [11]:
# 별 거 없으니 안녕 굿바이

data.drop('engine_unit', axis=1, inplace=True)

In [12]:
# 같은 방식으로 maxPower도 뒤에 쳐낼겁니다

data[['max_power', 'max_power_unit']] = data['max_power'].str.split(expand=True)

In [14]:
# 같은 방식으로 maxPower도 바꿔봅시다

data['max_power'] = data['max_power'].astype('float32')

ValueError: could not convert string to float: 'bhp'

In [15]:
# 저 놈 잡아라
data[data['max_power']=='bhp']

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


In [16]:
# 함수로 예외처리 한 번 해볼까?

def isFloat(value):
    try:
        num = float(value)
        return num
    except ValueError:
        return np.NaN

# 이 함수의 구조는 정상적이라면 float으로 바꾸겠지만 그렇지 않으면 결측치 처리하겠다는 얘기임

In [17]:
data['max_power'] = data['max_power'].apply(isFloat)

In [18]:
data['max_power_unit'].unique()

array(['bhp', nan, None], dtype=object)

In [19]:
# max_power 잘 쳐냈고 unit에도 뭐 별거 없으니 날려버립시다

data.drop('max_power_unit', axis=1, inplace=True)

In [20]:
# 마일리지도 같은 방식으로

data[['mileage', 'mileage_unit']] = data['mileage'].str.split(expand=True)
data['mileage'] = data['mileage'].astype('float32')

In [21]:
data['mileage_unit'].unique()

array(['kmpl', 'km/kg', nan], dtype=object)

In [22]:
# 마일리지는 함부러 쳐 낼 수가 없다. 단위가 다 의미가 있어보이네
# 이럴땐 연료를 봅시다

data['fuel'].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [23]:
# LPG랑 CNG는 kg 단위로 팔고
# Diesel이랑 Petrol은 리터 단위로 판다
# 아이고 이거도 함수로 만들어서 구분해야겠구나

def mile(x):
    if x['fuel'] == 'Petrol':
        return x['mileage'] / 80.43
    elif x['fuel'] == 'Diesel':
        return x['mileage'] / 73.56
    elif x['fuel'] == 'LPG':
        return x['mileage'] / 40.85
    else :
        return x['mileage'] / 44.23

# kg이나 리터 당 저 가격으로 판다고 해서 나눠줄 겁니다
# 그렇게 되면 단위가 달라도 1리터 당 주행 거리가 튀어나오겠죠    

In [24]:
# 처리 합시다
data['mileage'] = data.apply(mile, axis=1)

In [25]:
# 유닛은 고생했으니(?) 보내주고
data.drop('mileage_unit', axis=1, inplace=True)

In [26]:
# 문제는 토크입니다
data['torque'].head()

0              190Nm@ 2000rpm
1         250Nm@ 1500-2500rpm
2       12.7@ 2,700(kgm@ rpm)
3    22.4 kgm at 1750-2750rpm
4       11.5@ 4,500(kgm@ rpm)
Name: torque, dtype: object

In [27]:
# 일단 대소문자가 섞여있으니까 하나로 통일합시다

data['torque'] = data['torque'].str.upper()

In [28]:
# 그 다음에는 NM이나 KGM을 구분할 거고

def torque_unit(x):
    if 'NM' in str(x):
        return 'Nm'
    elif 'KGM' in str(x):
        return 'kgm'

In [29]:
# 제발 잘 분리되어라

data['torque_unit'] = data['torque'].apply(torque_unit)

In [30]:
data['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [31]:
# 돌다리도 두들겨보고 건너라고 했다
data['torque'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
8123    False
8124    False
8125    False
8126    False
8127    False
Name: torque, Length: 8128, dtype: bool

In [32]:
# torque_unit이 결측치일 때 그 라인의 torque 확인
data[data['torque_unit'].isna()]['torque'].unique()

array([nan, '250@ 1250-5000RPM', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [33]:
# 대체 왜 아직도 nan이 남아있는 거야
data['torque_unit'].fillna('Nm', inplace=True)

In [34]:
def split_num(x):
    x = str(x) # Change data type to String
    for i,j in enumerate(x):
        if j not in '0123456789.': # 인덱스에 이제 결측치가 없겠지 설마
            cut = i
            break
    return x[:cut] # 결측치 있으면 그 전 자리 까지만 출력됨

In [35]:
# torque에 split_num을 적용해서 결측치가 날아갔나 봅시다
data['torque'] = data['torque'].apply(split_num)

In [36]:
# 다행이도 결측치 잘 날아간 것으로 확인
data['torque']

0         190
1         250
2        12.7
3        22.4
4        11.5
        ...  
8123    113.7
8124       24
8125      190
8126      140
8127      140
Name: torque, Length: 8128, dtype: object

In [37]:
# 근데 타입이 object네? 난 float을 원해
data['torque'] = data['torque'].astype('float64')

ValueError: could not convert string to float: ''

In [39]:
# 아니 쟤는 또 왜 들어 있어
data['torque'] = data['torque'].replace('', np.NaN)

In [40]:
# 되라 제발
data['torque'] = data['torque'].astype('float64')

In [41]:
data['torque'].head(7)

0    190.00
1    250.00
2     12.70
3     22.40
4     11.50
5    113.75
6      7.80
Name: torque, dtype: float64

In [42]:
# 이제 단위를 맞출 겁니다

def torque_trans(x):
    if x['torque_unit'] == 'kgm':
        return x['torque'] * 9.8066
    else:
        return x['torque']

In [44]:
data['torque'] = data.apply(torque_trans, axis=1)

In [46]:
# 이제 토크 단위는 필요 없어
data.drop('torque_unit', axis=1, inplace=True)

In [47]:
# 그래서 지금까지의 상황을 다시 보면

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,0.318108,1248.0,74.00,190.00000,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,0.287384,1498.0,103.52,250.00000,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,0.220067,1497.0,78.00,124.54382,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,0.312670,1396.0,90.00,219.66784,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,0.200174,1298.0,88.20,112.77590,5.0


In [48]:
# 자동차도 겁나 많아보이는데 그냥 브랜드 수준에서 압축할 수는 없나?
# 브랜드만 한번 때봅시다

data['name'] = data['name'].str.split(expand=True)[0]

In [50]:
# 만약 여기에 브랜드가 아니라 엉뚱한 게 들어있다면 끔찍하겠지
data['name'].unique()

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

In [51]:
# 랜드 로버가 뒤에 짤렸네
data['name'] = data['name'].replace('Land', 'Land Rover')

In [52]:
# 그래서 결측치가...
data.isna().mean()

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine           0.027190
max_power        0.026575
torque           0.027313
seats            0.027190
dtype: float64

- mileage 이하 컬럼들은 자동차 스펙과 관련되어 있기 때문에
- 어쭙잖게 mean이나 이런 거로 처리하려고 들어가면 얘네가 노이즈를 만들 수도 있다 (저자가 그랬어)
- 어차피 뭐 저거 결측치 비율이 전체 데이터의 2% 정도 밖에 안 되니까 날리자!
- 실제 상황에서 뭐가 좋은지 모르겠다면 평균 넣어서 돌려보고, 지금처럼 없애서 돌려본 걸 비교하면 됨

In [53]:
data.dropna(inplace=True)
len(data)

7906

In [54]:
# 나머지 컬럼들을 더미로 만들어 볼까요

data = pd.get_dummies(data, columns=['name', 'fuel', 'seller_type', 'transmission', 'owner'], drop_first=True)

In [56]:
data.head()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0


In [57]:
# 역시 머신러닝은 전처리가 다 잡아먹어
# 이제 랜덤포레스트 해보자

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop('selling_price', axis=1), data['selling_price'], test_size=0.2, random_state=100)

In [59]:
# 랜덤포레스트 2종류입니다
# 분류할 때 보통 RandomForestClassifier
# 지금같이 연속형 변수가 있다면 RandomForestRegressor

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=100)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [60]:
# 잘 나왔을까

from sklearn.metrics import mean_squared_error
print(f'train_rmse:{mean_squared_error(y_train, train_pred)**0.5} test_rmse:{mean_squared_error(y_test, test_pred)**0.5}')

train_rmse:53531.41548125947 test_rmse:131855.18391308116


In [61]:
# 음... rmse가 0에 가까울 수록 좋은건데 저건... 미쳤는데?
# 검증을 해봐야겠어

from sklearn.model_selection import KFold

In [63]:
# 데이터 인덱스느 8127번까지 있는데 실제로는 7906개라서 인덱스 리셋함
data.reset_index(drop=True, inplace=True)

In [64]:
kf = KFold(n_splits=5)

X = data.drop('selling_price', axis=1)
y = data['selling_price']

for i, j in kf.split(X):
    print(i, j)

[1582 1583 1584 ... 7903 7904 7905] [   0    1    2 ... 1579 1580 1581]
[   0    1    2 ... 7903 7904 7905] [1582 1583 1584 ... 3160 3161 3162]
[   0    1    2 ... 7903 7904 7905] [3163 3164 3165 ... 4741 4742 4743]
[   0    1    2 ... 7903 7904 7905] [4744 4745 4746 ... 6322 6323 6324]
[   0    1    2 ... 6322 6323 6324] [6325 6326 6327 ... 7903 7904 7905]


In [65]:
# k-fold 하겠습니다

train_rmse_total = []
test_rmse_total = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestRegressor(random_state=100)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    train_rmse = mean_squared_error(y_train, train_pred) ** 0.5
    test_rmse = mean_squared_error(y_test, test_pred) ** 0.5

    train_rmse_total.append(train_rmse)
    test_rmse_total.append(test_rmse)

In [67]:
print('train_rmse:', sum(train_rmse_total)/5, 'test_rmse:', sum(test_rmse_total)/5)

train_rmse: 56553.836119114814 test_rmse: 142936.58918244042


- 근데 이게 맞아요. 교차검증을 해야 제대로된 값임을 알 수가 있었으니..